In [1]:
import os
os.makedirs('out', exist_ok=True)

### Google Colabを利用する場合は、次のセルの全行の#を1つ削除して実行する

In [2]:
## Google Colabを利用する場合
## 以下の#(コメントアウト）を削除して実行

#from google.colab import drive
#drive.mount('/content/drive')

## データフレームへの読み込み
## Google DriveのColab Notebooksにダウンロードしたフォルダ(marketing)をセットすることを想定
#iPath = '/content/drive/MyDrive/Colab Notebooks/marketing/Chapter_5/in/'
#iFile=iPath+'yogurt.csv'

## コード 5.10
#import pandas as pd
#import seaborn as sns

#df = pd.read_csv(iFile)
#df.head()


In [3]:
# コード 5.10
import pandas as pd
import seaborn as sns

df = pd.read_csv('in/yogurt.csv')
df.head()

,小分類コード,小分類名,商品名,売上数量,売上額,売上日,曜日,レジ番号,レシート番号,売上時間,単価
0,180201,プレ−ンホ−ムヨ−グルト,Y1,1,247,20180129,月,103,57582,134700,247.0
1,180201,プレ−ンホ−ムヨ−グルト,Y1,1,222,20180410,火,101,49854,111800,222.0
2,180201,プレ−ンホ−ムヨ−グルト,Y1,1,123,20180608,金,101,78436,190600,123.0
3,180201,プレ−ンホ−ムヨ−グルト,Y1,1,247,20181123,金,102,5236,102800,247.0
4,180201,プレ−ンホ−ムヨ−グルト,Y1,1,247,20180217,土,101,28748,164100,247.0


# 通常売価などの計算


In [4]:
# コード 5.11 通常売価の計算
# 単価の頻度が一番多い価格を通常価格とする
ds = df[['売上日', '商品名', '単価']]
ds = ds[~ds.duplicated()]

# value_counts()を使う方法(1.10以上のみ)
# ds = ds[['商品名','単価']].value_counts()
# regPrice = ds.groupby('商品名').head(1)

# groupbyを使う方法
ds = ds.groupby(['商品名', '単価']).size()
regPrice = ds.sort_values(ascending=False).groupby('商品名').head(1)
regPrice = regPrice.reset_index('単価')
regPrice.columns = ['通常価格', '頻度']
regPrice

,通常価格,頻度
商品名,,
Y4,247.0,281
Y10,197.0,236
Y6,157.0,213
Y1,247.0,206
Y5,157.0,202
Y7,97.0,196
Y8,238.0,187
Y11,197.0,183
Y9,147.0,155


In [5]:
# コード 5.12 日別の販売価格を選択
# 日別の販売価格（同一日に複数の価格設定があれば数量合計が最大の価格を利用する)
qtty = df.groupby(['売上日', '商品名', '単価'])[['売上数量']].sum()
dailyPrice = qtty.loc[qtty.groupby(['売上日', '商品名'])['売上数量'].idxmax()]
dailyPrice

売上数量
売上日      商品名 単価         
20180101 Y10 197.0     5
         Y12 138.0     6
         Y13 397.0     2
         Y2  138.0     3
         Y3  138.0     2
...                  ...
20181231 Y5  157.0     5
         Y6  157.0    16
         Y7  97.0      8
         Y8  238.0     1
         Y9  131.0     4

[3865 rows x 1 columns]

In [6]:
# コード 5.13 価格設定を表す項目の追加
# 通常売価、特売、値上げを表す「価格設定」項目作成
dat = dailyPrice.join(regPrice)
dat = dat.reset_index().rename(columns={'単価': '販売価格', '売上数量': '販売数量'})
dat.loc[dat['通常価格'] < dat['販売価格'], '価格設定'] = "値上げ"
dat.loc[dat['通常価格'] > dat['販売価格'], '価格設定'] = "特売"
dat.loc[dat['通常価格'] == dat['販売価格'], '価格設定'] = "通常売価"
dat

,売上日,商品名,販売価格,販売数量,通常価格,頻度,価格設定
0,20180101,Y10,197.0,5,197.0,236,通常売価
1,20180101,Y12,138.0,6,147.0,149,特売
2,20180101,Y13,397.0,2,397.0,26,通常売価
3,20180101,Y2,138.0,3,147.0,135,特売
4,20180101,Y3,138.0,2,147.0,140,特売
...,...,...,...,...,...,...,...
3860,20181231,Y5,157.0,5,157.0,202,通常売価
3861,20181231,Y6,157.0,16,157.0,213,通常売価
3862,20181231,Y7,97.0,8,97.0,196,通常売価
3863,20181231,Y8,238.0,1,238.0,187,通常売価


In [7]:
# コード5.14 通常売価の平均販売数量の計算
# 通常売価のときの平均数量を計算
avgNum = dat[dat['価格設定'] == '通常売価'].groupby(
    ['商品名'], as_index=False)['販売数量'].mean()
avgNum = avgNum.rename(columns={'販売数量': '通常数量'})
rsl = dat.merge(avgNum, on='商品名')
rsl.head()

#rsl.to_csv('out/dat.csv', index=False)

,売上日,商品名,販売価格,販売数量,通常価格,頻度,価格設定,通常数量
0,20180101,Y10,197.0,5,197.0,236,通常売価,18.894444
1,20180102,Y10,188.0,20,197.0,236,特売,18.894444
2,20180103,Y10,197.0,30,197.0,236,通常売価,18.894444
3,20180104,Y10,207.0,15,197.0,236,値上げ,18.894444
4,20180105,Y10,207.0,17,197.0,236,値上げ,18.894444


# 価格弾力性の計算

In [8]:
# コード 5.15 価格弾力性の計算
# 通常価格以外の価格について価格弾力性を計算する
df = rsl[rsl['価格設定'] != '通常売価'].copy()

# 弾力性の価格変化率として利用(midpoint methodを利用)
df['価格変化率'] = (df['販売価格']-df['通常価格'])/((df['販売価格']+df['通常価格'])/2)
df['数量変化率'] = (df['販売数量']-df['通常数量'])/((df['販売数量']+df['通常数量'])/2)
df['価格弾力性'] = df['数量変化率']/df['価格変化率']
display(df.head())
df.to_csv("out/elasticity.csv", index=False)

,売上日,商品名,販売価格,販売数量,通常価格,頻度,価格設定,通常数量,価格変化率,数量変化率,価格弾力性
1,20180102,Y10,188.0,20,197.0,236,特売,18.894444,-0.046753,0.056849,-1.215937
3,20180104,Y10,207.0,15,197.0,236,値上げ,18.894444,0.049505,-0.229798,-4.641928
4,20180105,Y10,207.0,17,197.0,236,値上げ,18.894444,0.049505,-0.105556,-2.132240
5,20180106,Y10,207.0,22,197.0,236,値上げ,18.894444,0.049505,0.151882,3.068007
6,20180107,Y10,207.0,16,197.0,236,値上げ,18.894444,0.049505,-0.165897,-3.351122


In [9]:
# コード 5.16 価格弾力性の平均値

# 価格弾力性の平均値
avgEl = df.groupby(["商品名", "価格設定"], as_index=False)["価格弾力性"].mean()
avgEl.head(30)

,商品名,価格設定,価格弾力性
0,Y1,特売,-3.218192
1,Y10,値上げ,-4.756690
2,Y10,特売,-8.631454
3,Y11,値上げ,-7.727878
4,Y11,特売,3.057336
5,Y12,特売,-5.978409
6,Y13,特売,-0.045118
7,Y2,特売,-0.992598
8,Y3,特売,-4.760052
9,Y4,特売,-5.134526


# 交差弾力性の計算

In [10]:
# コード 5.17 交差弾力性の計算
df = pd.read_csv("out/elasticity.csv")

df = df[["商品名", "売上日", "価格変化率", "数量変化率"]]
df2 = df

# 交差弾力性を調べるために同じ日付の別商品の変化量をすべて結合
df = pd.merge(df, df2, on="売上日", how="outer")
df = df[df["商品名_x"] != df["商品名_y"]]

# 交差弾力性の計算 yの数量変化率/xの価格変化率 (xの価格が変化したときにyの数量がどの程度動くか)
df['交差弾力性'] = df['数量変化率_y']/df['価格変化率_x']
display(df.head())
df.to_csv("out/cross_elasticity.csv", index=False)

,商品名_x,売上日,価格変化率_x,数量変化率_x,商品名_y,価格変化率_y,数量変化率_y,交差弾力性
1,Y10,20180102,-0.046753,0.056849,Y12,-0.063158,0.590205,-12.623831
2,Y10,20180102,-0.046753,0.056849,Y2,-0.169742,0.056604,-1.210692
3,Y10,20180102,-0.046753,0.056849,Y3,-0.063158,0.338902,-7.248740
4,Y10,20180102,-0.046753,0.056849,Y4,-0.106610,0.158273,-3.385292
5,Y10,20180102,-0.046753,0.056849,Y7,-0.108696,0.178361,-3.814936


In [11]:
# コード 5.18 価格変化率 y の昇順の結果
# yの価格変化率が小さい順に並び替え
df['abs_py'] = df['価格変化率_y'].abs()
df = df.sort_values("abs_py", ascending=True)
df.head()

# 交差弾力性の解釈
# 交差弾力性はxの価格を変化させたときに、yの数量変化の程度を示す。
# ただし、xとyの価格が両方変化している場合には意味解釈も異なるように思う
# xだけ値下げ、xとyも値下げ、xを値上げ,xとyを値上げ

,商品名_x,売上日,価格変化率_x,数量変化率_x,商品名_y,価格変化率_y,数量変化率_y,交差弾力性,abs_py
15239,Y9,20181219,-0.722222,1.010403,Y5,0.002545,0.148541,-0.205672,0.002545
15227,Y3,20181219,-0.138182,0.576687,Y5,0.002545,0.148541,-1.074969,0.002545
15233,Y7,20181219,-0.675862,0.264292,Y5,0.002545,0.148541,-0.219780,0.002545
15221,Y2,20181219,-0.063158,-0.345351,Y5,0.002545,0.148541,-2.351901,0.002545
15215,Y12,20181219,-0.138182,0.239415,Y5,0.002545,0.148541,-1.074969,0.002545
